In [1]:
from typing import List, Set, Tuple
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [2]:
excel_path  = r"C:\Users\issam\Desktop\Files\PhD\DRPRL\data\StaticProblemInstances\GroupA\distancematrix_with_20_nodes(seconds).xlsx"
D1 = pd.read_excel(excel_path, sheet_name='D1', header = None).values


c:\Users\issam\Desktop\Files\PhD\DRPRL\venv\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Model

In [3]:
model = gp.Model("DRP")
model.setParam('TimeLimit', 60)

Restricted license - for non-production use only - expires 2026-11-23
Set parameter TimeLimit to value 60


Variables

In [4]:
n = len(D1)
x = {}
for i in range(n):
    for j in range(n):  # Only upper triangular
        if i != j:
            x[i,j] = model.addVar(vtype=GRB.BINARY, name=f'x_{i}_{j}')

In [5]:
# Print variable count
print(f"Number of variables: {len(x)}")

Number of variables: 600


objective function 

In [6]:
obj = gp.quicksum(D1[i,j]*x[i,j] for i,j in x)
model.setObjective(obj, GRB.MINIMIZE)

constraints <br> flow conservation

In [7]:
for h in range(20):
    model.addConstr(gp.quicksum(x[i,h] for i in range(n) if i != h) == 
                    gp.quicksum(x[h,j] for j in range(n) if h != j))

visit each customer once 

In [8]:
for h in range(20) :
    model.addConstr(gp.quicksum(x[i,j] for i in range(n) if i!=j) == 1)

Solve

In [9]:
model.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13620H, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 40 rows, 600 columns and 1440 nonzeros
Model fingerprint: 0xfc74259d
Variable types: 0 continuous, 600 integer (600 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3122.2386778
Presolve removed 19 rows and 19 columns
Presolve time: 0.01s
Presolved: 21 rows, 581 columns, 981 nonzeros
Variable types: 0 continuous, 581 integer (581 binary)

Root relaxation: objective 1.329260e+02, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [ ]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal objective: {model.objVal}")
    print("\nSelected arcs:")
    for i,j in x:
        if x[i,j].X > 0.5:
            print(f"{i} -> {j} (cost: {D1[i,j]})")
else: 
    print("No solution found")